# Dashboard

In [54]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from multiprocessing import Pool
from IPython.core import display as ICD
from functools import partial
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import pairwise_distances


In [55]:
def extract_info(name):
    frequency,support,*group,other=name.split("-")
    return frequency,support,group[0][1:]+'-'+group[1][:-1]

files = !ls *groups.dat
group_stats = [] 
columns = ["file_name","frequency","support","group_size","nb_groups"]
stats = []
for i in files :
    df = pd.read_csv(i,index_col=0)
    stats.append((i,df.group.unique().shape,df.period.unique()))
pd.DataFrame(stats,columns=["file","nb_groups","groups_periods"])

/home/abdelwahab/.virtualenvs/3env/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,file,nb_groups,groups_periods
0,3M-20-[2-100]-groups.dat,"(2702,)","[1996-08, 1996-11]"
1,3M-20-[2-10]-groups.dat,"(2702,)","[1996-08, 1996-11]"
2,3M-20-[2-20]-groups.dat,"(2702,)","[1996-08, 1996-11]"
3,3M-20-[2-5]-groups.dat,"(2551,)","[1996-08, 1996-11]"
4,3M-30-[2-100]-groups.dat,"(18,)",[1996-08]
5,3M-30-[2-10]-groups.dat,"(18,)",[1996-08]
6,3M-30-[2-20]-groups.dat,"(18,)",[1996-08]
7,3M-30-[2-5]-groups.dat,"(18,)",[1996-08]
8,6M-20-[2-100]-groups.dat,"(12725,)","[1996-08, 1997-02]"
9,6M-20-[2-10]-groups.dat,"(12725,)","[1996-08, 1997-02]"


#### Set of function to recompute the groups, with the following approach:
1. Regroup similar groups of each period to new groups according to jaccard score
2. 

In [56]:
def jaccard_similarity(a, b):
    """Compute Jaccard distance between two arrays
    Inputs :
        a: n element array
        b: m element array
    """
    return len(a[0].intersection(b[0]))/len(a[0].union(b[0]))


def similar_groups_union(groups,threashold):
    """
    Create new groups from given ones,
    each group is affected to the new group having the minimal jaccard distance with (comparing to all new groups)
    if the minimal jaccard distance found is under the threashold, the group is added as a new group 
    """
    new_groups = [groups[0]]
    for group in groups[1:]:
        distances = np.array([len(group[0].intersection(i[0]))/len(group[0].union(i[0])) for i in new_groups])
        for idx,i in enumerate(new_groups):
            distance = 1-len(group[0].intersection(i[0]))/len(group[0].union(i[0]))
            if distance<threashold:
                new_groups[idx][0] = new_groups[idx][0].union(group[0])
                break
            new_groups.append(group)
            
    return new_groups    

def recompute_groups(df,threashold):
    """
    Regroup similar groups of each period to new groups according to the Jaccard distance between them
    Input:
        df: DataFrame 
        threashold: Minimal Jaccard distance under which two groups are supposed similar
    Output:
        Dataframe 
    """
    groups = pd.DataFrame()
    for period,i in df.groupby("period"):
        groups =  [[set(x.split(","))] for x in i.users_ids.values]
        res = pd.DataFrame(similar_groups_union(groups,threashold))
        res["period"] = period
        groups = pd.concat([groups,res],axis=0,sort=False)
    # TODO exclude nan from concatenated list
    new_df = groups.drop("period",axis=1).apply(lambda x : ",".join(str(z) for z in x),axis=1).to_frame()
    new_df["period"]= groups.period
    new_df.columns = ["users","period"]
    return new_df

In [ ]:
def compute(i,threashold=0.4):
    df = pd.read_csv(i,index_col=0)
    df_computed = recompute_groups(df,threashold)
    df_computed.to_csv("C"+i)
with Pool(8) as pool:
    pool.map(compute,["Y-20-[2-100]-groups.dat","Y-2-[2-10]-groups.dat","Y-2-[2-5]-groups.dat"])

In [ ]:
!ls results/

In [ ]:
df = pd.read_csv("CY-2-[2-5]-groups.dat",index_col=0)
lb = MultiLabelBinarizer()
a = lb.fit_transform(df.users.map(lambda x : [int(i) for i in x.split(",") if i!="nan"]))
a = pd.DataFrame(pairwise_distances(a,metric="jaccard"),index=df.index,columns=df.index).stack().reset_index()
a[(a[0]>0)&(a[0]<1)&(a.level_0<a.level_1)].groupby(["level_0","level_1"])[0].count().reset_index()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 1,
      thickness = 1,
      line = dict(color = "black", width = 0.5),
      label = a.index,
      color = "blue"
    ),
    link = dict(
      source = out.level_0, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = out.level_1,
      value = out[0]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=1)
fig.show()

In [ ]:
.

# DRAFT 

In [1]:
import numpy as np
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer
from sys import getsizeof
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist
import numpy as np
import pandas as pd

lb = MultiLabelBinarizer()
# Minimal distance threashold
threashold = 0.2

In [2]:
for period,i in df.groupby("period"):
    pass
groups =  [[set(x.split(","))] for x in i.users_ids.values]




def similar_groups_union(groups,threashold):
    new_groups = [groups[0]]
    for group in groups[1:]:
        
#         distances = cdist([group],new_groups,metric=jaccard_similarity)[0]<threashold
        for i in np.where(distances)[0]:
            new_groups[i][0] = new_groups[i][0].union(group[0])
        if np.where(distances)[0].shape[0]==0:
            new_groups.append(group)
    return new_groups    

similar_groups_union(groups,0.4)

NameError: name 'df' is not defined

In [4]:
a = ["1","2","22"]
b = ["11",22,"2"]
a = set(a)
len(a.intersection(b))/len(a.union(b))

0.2

In [99]:
def jaccard_similarity(a, b):
    """Compute Jaccard distance between two arrays
    Inputs :
        a: n element array
        b: m element array
    """
    a = set(a)
    return len(a.intersection(b))/len(a.union(b))

def compute_distances(a,b):
    return np.array([[jaccard_similarity(a,i)] for i in b])

# TODO CUrrent version affectiong group to the first group having the minimal jaccard distance threashold
# TODO the groups have to be added to all groups having distance under threashold
def similar_groups_union(groups,threashold):
    new_groups = [groups[0]]
    for i in groups[1:]:
        found = False 
        for idx,g in enumerate(new_groups):
            if jaccard_similarity(i,g)<threashold:
                found = True
                new_groups[idx] = np.union1d(g,i)
                break
        if not found:
            new_groups.append(i) 
    return new_groups    

def recompute_groups(df,threashold):
    """
    Regroup similar groups of each period to new groups according to the Jaccard distance between them
    Input:
        df: DataFrame 
        threashold: Minimal Jaccard distance under which two groups are supposed similar
    Output:
        Dataframe 
    """
    groups = pd.DataFrame()
    for period,i in df.groupby("period"):
        res = pd.DataFrame(similar_groups_union(i.users_ids.str.split(",",expand=True).dropna(axis=1).values,threashold))
        res["period"] = period
        groups = pd.concat([groups,res],axis=0,sort=False)
    new_df = groups.drop("period",axis=1).apply(lambda x : ",".join(str(z) for z in x),axis=1).to_frame()
    new_df["period"]= groups.period
    new_df.columns = ["users","period"]
    return new_df

In [21]:
def similar_groups(df,threashold=0.2)
    results = []
    for period,i in df.groupby("period"):
        groups = i.users_ids.str.split(",",expand=True).dropna(axis=1).values.astype(str)  
        new_groups = [groups[0]]

        for group in groups[1:]:
            distance = compute_distances(group,new_groups)
            idx = distance<threashold
            if idx.any():
                print("append")
                for idxx in np.where(idx)[0]:
                    prev= new_groups.copy()
                    new_groups[idxx] = np.union1d(new_groups[idxx],group)
            else:
                print("add",len(new_groups))
                new_groups.append(group)
        results.append((new_groups,period))   
        df = pd.DataFrame()
        for group,period in results:
            group = pd.DataFrame(group)
            group["period"]= period
            df = pd.concat([df,group],axis=0,sort=True)
    return df

add 1
add 2
add 3
add 4
add 5
add 6
add 7
add 8
add 9
add 10
add 11
add 12
add 13
add 14
add 15
add 16
add 17
add 18
add 19
add 20
add 21
add 22
add 23
add 24
add 25
add 26
add 27
add 28
add 29
add 30
add 31
add 32
add 33
add 34
add 35
add 36
add 37
add 38
add 39
add 40
add 41
add 42
add 43
add 44
add 45
add 46
add 47
add 48
add 49
add 50
add 51
add 52
add 53
add 54
add 55
add 56
add 57
add 58
add 59
add 60
add 61
add 62
add 63
add 64
add 65
add 66
add 67
add 68
add 69
add 70
add 71
add 72
add 73
add 74
add 75
add 76
add 77
add 78
add 79
add 80
add 81
add 82
add 83
add 84
add 85
add 86
add 87
add 88
add 89
add 90
add 91
add 92
add 93
add 94
add 95
add 96
add 97
add 98
add 99
add 100
add 101
add 102
add 103
add 104
add 105
add 106
add 107
add 108
add 109
add 110
add 111
add 112
add 113
add 114
add 115
add 116
add 117
add 118
add 119
add 120
add 121
add 122
add 123
add 124
add 125
add 126
add 127
add 128
add 129
add 130
add 131
add 132
add 133
add 134
add 135
add 136
add 137
add 138
add 

append
add 975
append
add 976
add 977
add 978
add 979
add 980
add 981
append
add 982
add 983
add 984
add 985
add 986
append
add 987
add 988
add 989
add 990
add 991
add 992
add 993
add 994
append
add 995
add 996
add 997
add 998
add 999
add 1000
add 1001
add 1002
add 1003
add 1004
add 1005
append
add 1006
add 1007
add 1008
add 1009
add 1010
add 1011
add 1012
add 1013
add 1014
add 1015
add 1016
add 1017
add 1018
add 1019
add 1020
add 1021
append
add 1022
add 1023
add 1024
add 1025
add 1026
add 1027
add 1028
add 1029
add 1030
add 1031
append
add 1032
add 1033
add 1034
add 1035
add 1036
add 1037
add 1038
add 1039
append
append
add 1040
add 1041
add 1042
add 1043
add 1044
add 1045
add 1046
add 1047
append
add 1048
add 1049
add 1050
add 1051
append
append
add 1052
append
add 1053
add 1054
add 1055
append
add 1056
add 1057
add 1058
add 1059
add 1060
add 1061
add 1062
add 1063
add 1064
add 1065
add 1066
add 1067
add 1068
add 1069
add 1070
add 1071
add 1072
add 1073
add 1074
add 1075
add 1076
ad

add 1843
add 1844
add 1845
add 1846
add 1847
add 1848
add 1849
add 1850
add 1851
add 1852
add 1853
add 1854
add 1855
add 1856
add 1857
add 1858
add 1859
add 1860
add 1861
add 1862
add 1863
add 1864
add 1865
append
add 1866
add 1867
add 1868
append
append
add 1869
add 1870
add 1871
add 1872
add 1873
add 1874
add 1875
append
add 1876
append
add 1877
add 1878
add 1879
add 1880
add 1881
add 1882
add 1883
add 1884
add 1885
add 1886
add 1887
add 1888
add 1889
add 1890
add 1891
add 1892
add 1893
add 1894
add 1895
add 1896
add 1897
add 1898
append
add 1899
add 1900
add 1901
add 1902
add 1903
add 1904
add 1905
add 1906
add 1907
add 1908
add 1909
add 1910
add 1911
add 1912
add 1913
add 1914
add 1915
add 1916
add 1917
add 1918
add 1919
add 1920
add 1921
add 1922
add 1923
add 1924
add 1925
add 1926
add 1927
add 1928
add 1929
add 1930
add 1931
add 1932
add 1933
add 1934
add 1935
add 1936
add 1937
add 1938
add 1939
add 1940
add 1941
add 1942
add 1943
add 1944
add 1945
add 1946
add 1947
add 1948
add 

add 2684
add 2685
add 2686
add 2687
add 2688
add 2689
add 2690
add 2691
add 2692
add 2693
add 2694
append
add 2695
add 2696
add 2697
add 2698
append
add 2699
add 2700
add 2701
add 2702
add 2703
add 2704
add 2705
add 2706
add 2707
add 2708
append
add 2709
add 2710
add 2711
add 2712
append
add 2713
add 2714
add 2715
add 2716
add 2717
add 2718
add 2719
add 2720
add 2721
add 2722
add 2723
add 2724
add 2725
add 2726
add 2727
add 2728
add 2729
add 2730
add 2731
add 2732
add 2733
add 2734
add 2735
add 2736
add 2737
add 2738
add 2739
add 2740
add 2741
add 2742
add 2743
add 2744
add 2745
add 2746
add 2747
add 2748
add 2749
add 2750
add 2751
add 2752
add 2753
add 2754
add 2755
add 2756
add 2757
add 2758
add 2759
add 2760
add 2761
add 2762
add 2763
add 2764
add 2765
add 2766
add 2767
add 2768
add 2769
add 2770
add 2771
add 2772
add 2773
add 2774
append
add 2775
add 2776
add 2777
add 2778
add 2779
add 2780
add 2781
add 2782
append
append
add 2783
append
append
append
append
append
add 2784
add 278

append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append
append

KeyboardInterrupt: 

In [308]:
df = pd.DataFrame()
for group,period in results:
    group = pd.DataFrame(group)
    group["period"]= period
    df = pd.concat([df,group],axis=0,sort=True)

In [51]:
users = lb.fit_transform(df.sort_values("group").users_ids.map(lambda x:[int(i) for i in x.split(",")])).astype(bool)

In [49]:
from sklearn.metrics import jaccard_score
threashold = 0,2
lb = MultiLabelBinarizer()

In [52]:
results = pd.DataFrame()
for period,i in df.groupby("period"):
    groups = np.array([])
    for idx,group in zip(i.index,users[i.index]):
        if groups.shape[0]==0:
            groups = np.array([group],dtype=np.bool)
            continue
        
        group = np.array([group],dtype=np.bool)
        distance = pairwise_distances(group,groups,metric="jaccard",n_jobs=-1)[0]
        i = distance<0.5
        if i.any():
            groups[i]+=group
        else:
            groups = np.append(groups,group,axis=0)
    groups = pd.DataFrame(groups)
    groups["period"]=period
    results = pd.concat([results,groups],sort=False)        

KeyboardInterrupt: 

""


In [ ]:
def count_similarities(path1, path2):
    return np.sum(path1.astype(bool) & path2.astype(bool))

In [ ]:
prev = None
prev_period = None
out  = pd.DataFrame()
a = results.reset_index()
for period,i in a.groupby("period"):
    if prev is None:
        prev=i.drop("period",axis=1)
        prev_period = period
        continue
    comparison = pd.DataFrame(pairwise_distances(prev,i.drop("period",axis=1),metric=count_similarities),columns=i.index,index=prev.index)
    out = pd.concat([out,comparison[comparison>0].stack().reset_index()])

In [ ]:
a.index

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 1,
      thickness = 1,
      line = dict(color = "black", width = 0.5),
      label = a.index,
      color = "blue"
    ),
    link = dict(
      source = out.level_0, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = out.level_1,
      value = out[0]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=1)
fig.show()

In [334]:
results
mask = np.ones(users.shape[0],bool)
results = pd.DataFrame()
users = users.astype(bool)
for idx,i in df.groupby("period"):
    mask[i.index]=False
    try:
        a = pd.DataFrame(pairwise_distances(users[i.index],users[mask],metric="jaccard"))
        a.columns = df.index[mask]
        a.index = i.index
        a = a[a<1].stack().reset_index()
#         a = a[a<1].stack().reset_index()
        results = pd.concat([results,a],axis=0)
    except ValueError as e : #Exception for last period having no next 
        print(e)


,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,period
0,False,False,True,True,True,True,False,True,True,False,...,False,False,True,True,True,True,True,True,False,1996-08
1,False,False,True,True,True,True,False,True,True,False,...,False,False,True,True,True,True,True,True,False,1996-08
2,True,True,False,False,False,False,True,False,False,True,...,True,True,False,False,False,False,False,False,True,1996-08
0,False,False,True,False,False,False,False,False,True,False,...,False,False,True,True,True,True,True,False,False,1997-02


In [255]:
group = users[0]
groups = np.array([group],dtype=np.bool)
group = np.array([users[1]],dtype=np.bool)
distance = pairwise_distances(group,groups,metric="jaccard")[0]
i = distance<0.2

if i.any():
    groups[i]+=group
else:
    groups = np.append(groups,[group])


In [109]:
groups = np.array([group])
groups = np.append(groups,[users[2]],axis=0)
distances = pairwise_distances(np.array([group]),groups,metric="jaccard")
pd.DataFrame(distances).stack().reset_index()

,level_0,level_1,0
0,0,0,0.0
1,0,1,1.0


### Unifing similar groups 

In [ ]:
from sklearn.metrics import pairwise_distances
prev = None
nb_split = 20
distances = []
for period,i in df.groupby("period"):
    for idx,ii in zip(np.array_split(i.index,nb_split),np.array_split(users[i.index],nb_split)):
        a = pairwise_distances(ii,metric="jaccard",n_jobs=-1)
        break
    break
#     pairwise_distances(users[i.index],metric="jaccard",n_jobs=-1)
#         distance = pd.DataFrame(distance)
#         distance = distance[distance<threashold]
        

In [ ]:
a = pd.DataFrame(a,columns=idx,index=idx)
a.query("value>0.2")

In [ ]:
a[a<0.2]

In [ ]:
mask = np.ones(users.shape[0],bool)
results = pd.DataFrame()
users = users.astype(bool)
for idx,i in df.groupby("period"):
    mask[i.index]=False
    try:
        a = pd.DataFrame(pairwise_distances(users[i.index],users[mask],metric="jaccard"))
        a.columns = df.index[mask]
        a.index = i.index
        a = a[a<1].stack().reset_index()
#         a = a[a<1].stack().reset_index()
        results = pd.concat([results,a],axis=0)
    except ValueError as e : #Exception for last period having no next 
        print(e)


In [ ]:
df.group.size
# 592811 group

In [ ]:
a = lb.fit_transform(df.users_ids.map(lambda x : [int(i) for i in x.split(",")]))
a

In [ ]:
X
[np.flatnonzero(d < r) for d in D_chunk]

In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances_chunked
def reduce_func(D_chunk, start):
    print(start)
    return D_chunk
gen = pairwise_distances_chunked(users, reduce_func=reduce_func,metric="jaccard")

In [ ]:
for i in gen:
    break

In [ ]:
from sklearn.metrics import pairwise_distances_chunked


In [ ]:
prev = None
for period,i in pd.DataFrame(a).join(df.period).groupby("period"):
    if prev is None:
        prev = i
        continue
    pass

In [ ]:
a = np.array([[0,1,1,1],[1,1,2,3]])
b = np.array([[1,0,0,0],[0,1,1,2]])
prod = a*b
print(prod)
prod.sum()
np.prod(a,b)

# Compute distance 

#### Apply MultilabelBinarizer, and add period and group for each row

In [ ]:
users = lb.fit_transform(df.users_ids.map(lambda x:[int(i) for i in x.split(",")]))
a = pd.DataFrame(users).reset_index().merge(df[["period","group"]],left_on="index",right_on="group")
a.drop(["group","index"],axis=1,inplace=True)

users = users.astype(bool)

#### Compute distance of each group from period[i] with all groups having period[j] j>i 

In [ ]:
mask = np.ones(users.shape[0],bool)
results = pd.DataFrame()
users = users.astype(bool)
for idx,i in df.groupby("period"):
    mask[i.index]=False
    try:
        a = pd.DataFrame(pairwise_distances(users[i.index],users[mask],metric="jaccard"))
        a.columns = df.index[mask]
        a.index = i.index
        a = a[a<1].stack().reset_index()
#         a = a[a<1].stack().reset_index()
        results = pd.concat([results,a],axis=0)
    except ValueError as e : #Exception for last period having no next 
        print(e)


In [ ]:
a = results[results[0]<1]
a

In [ ]:
results.columns = ["group","group_y","value"]
stats = results.astype(int).merge(df[["users_ids","group"]],left_on="group",right_on="group").merge(df[["users_ids","group"]],left_on="group_y",right_on="group")
stats["users_ids_x"].apply(lambda x:print([int(i) for i in x)]))

### Compute distances matrix: 
1. MultilabelBinarizer
2. Pairwise distances using Jaccard distance  as a metric

In [ ]:
users = lb.fit_transform(df.users_ids.map(lambda x:[int(i) for i in x.split(",")]))
distances = pairwise_distances(users,metric='jaccard')
distances = pd.DataFrame(distances)
distances.head()

In [ ]:
# Minimal distance threashold
threashold = 0.2

In [ ]:
similarities = distances[(distances<threashold)&(distances>0)].stack().reset_index()
similarities
similarities.columns = ["group_x","group_y","value"]
similarities = similarities.merge(df,left_on="group_x",right_on="group")
similarities.period.unique()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = similarities.group_x.tolist(),
      color = "blue"
    ),
    link = dict(
      source = similarities.group_x.tolist(), # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = similarities.group_y.tolist(),
      value = similarities.value.tolist()
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
df.users_ids.str.split(",",expand=True).dropna()

In [ ]:
df[(df.group==0)|(df.group==3)]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = similarities_in_groups.group_id_x.tolist(),
      color = "blue"
    ),
    link = dict(
      source = similarities_in_groups.group_id_x.tolist(), # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = similarities_in_groups.group_id_y.tolist(),
      value = similarities_in_groups.period_x.tolist()
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
df[df.group_id==3]

In [ ]:
a = pd.DataFrame(comparison_result)
import seaborn as sns 
f, ax = plt.subplots(figsize=(20, 20))
# corr =bank.corr()
sns.heatmap(a)

In [ ]:
periods = groups_details.period.unique()
results = []
for (period,group_id),i in groups_details.groupby(["period","group_id"]):
    try:
        next_period = periods[periods>period][0]
    except:
        continue
    groups_to_compare = users[groups_details[groups_details.period==next_period]["group_id"].index]
    size = groups_to_compare.shape[0]
    
    group_to_compare = np.repeat([users[group_id]],size,axis=1)
    
    comparison_result = pairwise_distances(group_to_compare,group_to_compare,metric='jaccard')
    results.append((group_id,comparison_result))
period = 2016
results
# users

In [ ]:
groups_details = df.groupby(["period","group_id"])["user_id"].apply(lambda x: [z for z in x ] ).reset_index(name="users")
a = groups_details.users[0]
# groups_details.groupby(["period","users","group_id"])["users"].to_frame()
for i in groups_details.loc[df.group_id.drop_duplicates().reindex()]:
#     a = [int(z) for z in i["users"].split(" ")]
    print(i)
#     pass
# groups_details.group_id.nunique()
# c = b.copy()
# for i in range(100):
#     c[0][i]=1
# print(jaccard_score([],c,average=None))
# b
groups_details.group_id.value_counts()
users = groups_details.groupby("group_id").first().reset_index()["users"]
# b = pd.DataFrame(data=lb.fit_transform(a.users))
print(jaccard_score(users,users,average=None))


In [ ]:
from scipy.sparse import lil_matrix
def label_to_sm(labels, n_classes):
    sm = lil_matrix((len(labels), n_classes))
    for i, label in enumerate(labels):
        sm[i, label] = 1
    return sm
label_to_sm(users[0],1000).toarray()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
lb = preprocessing.MultiLabelBinarizer()
a = df.groupby(["period","group_id"])["user_id"].apply(lambda x: [z for z in x]).reset_index(name="users")
b = pd.DataFrame(data=lb.fit_transform(a.users))
# a["users"].value_counts(ascending=True)
# ICD.display(df.groupby(["group_id"]).user_id.count().reset_index(name="nb_users").groupby('nb_users').group_id.count().reset_index(name="nb_groups").sum())
b

In [ ]:
from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=100)
# kmeans.fit(b)
# labels = kmeans.predict(b)
# a["clusters"] = labels
# a["clusters"].value_counts().reset_index(name="count_values")
# k means determine k
# X=b
# distortions = []
# K = range(10,100)
# for k in K:
#     kmeanModel = KMeans(n_clusters=k).fit(X)
#     kmeanModel.fit(X)
#     distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
# La moyenne du nombre d'utilisateur/groupe 
ICD.display(df.groupby(["group_id"]).user_id.count().reset_index(name="nb_users").groupby('nb_users').group_id.count().reset_index(name="nb_groups").sum())
# plt.hist(df.groupby(["period","group_id"]).user_id.count())
# nb_users     135
# nb_groups    536


In [ ]:
group_stats[group_stats.dataset_filter=="rating>4"].sort_values(["frequency","support"],ascending=False)

In [ ]:
def extract_info(name):
    frequency,support,*group,dataset_filter,other=name.split("-")
    frequency = frequency.split("/")[1]
    return frequency,support,group[0][1:]+'-'+group[1][:-1],dataset_filter

files = !ls *groups.dat
group_stats = [] 
columns = ["file_name","frequency","support","group_size","dataset_filter","nb_groups","nb_users"]
for i in files :
    df = pd.read_csv(i,index_col=0)
    nb_users,nb_groups = df[["users_ids","group"]].nunique()
    user_per_group = df.groupby(["group"]).users_ids.count().reset_index(name="nb_users").groupby('nb_users').group.count().reset_index(name="nb_groups")
    group_stats.append((i,*extract_info(i),nb_groups,nb_users))
group_stats = pd.DataFrame(group_stats,columns=columns).sort_values("nb_groups")
ICD.display(group_stats.head(40))
ICD.display(group_stats.tail(10))

In [ ]:
group_stats["rank"] = group_stats.nb_groups/group_stats.nb_users
group_stats.sort_values("rank",ascending=False)

In [ ]:
# Import Data
df = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/mpg_ggplot2.csv")
df_counts = df.groupby(['hwy', 'cty']).size().reset_index(name='counts')

# Draw Stripplot
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)    
sns.stripplot(df_counts.cty, df_counts.hwy, size=df_counts.counts*2, ax=ax)

# Decorations
plt.title('Counts Plot - Size of circle is bigger as more points overlap', fontsize=22)
plt.show()

In [ ]:
import numpy as np 
data = {'a': np.arange(50),
        'c': np.random.randint(0, 50, 50),
        'd': np.random.randn(50)}
data['b'] = data['a'] + 10 * np.random.randn(50)
data['d'] = np.abs(data['d']) * 100

plt.scatter('a', 'b', c='c', s='d', data=data)
plt.xlabel('entry a')
plt.ylabel('entry b')
plt.show()

In [ ]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
 
# y-axis in bold
rc('font', weight='bold')
 
# Values of each group
bars1 = [12, 28, 1, 8, 22]
bars2 = [28, 7, 16, 4, 10]
bars3 = [25, 3, 23, 25, 17]
 
# Heights of bars1 + bars2
bars = np.add(bars1, bars2).tolist()
 
# The position of the bars on the x-axis
r = [0,1,2,3,4]
 
# Names of group and bar width
names = ['A','B','C','D','E']
barWidth = 1
 
# Create brown bars
plt.bar(r, bars1, color='#7f6d5f', edgecolor='white', width=barWidth)
# Create green bars (middle), on top of the firs ones
plt.bar(r, bars2, bottom=bars1, color='#557f2d', edgecolor='white', width=barWidth)
# Create green bars (top)
plt.bar(r, bars3, bottom=bars, color='#2d7f5e', edgecolor='white', width=barWidth)
 
# Custom X axis
plt.xticks(r, names, fontweight='bold')
plt.xlabel("group")
 
# Show graphic
plt.show()


In [ ]:
import seaborn as sns
sns.set()
df.set_index('App').T.plot(kind='bar', stacked=True)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

#Read in data & create total column
stacked_bar_data = pd.read_csv("C:\stacked_bar.csv")
stacked_bar_data["total"] = stacked_bar_data.Series1 + stacked_bar_data.Series2

#Set general plot properties
sns.set_style("white")
sns.set_context({"figure.figsize": (24, 10)})

#Plot 1 - background - "total" (top) series
sns.barplot(x = stacked_bar_data.Group, y = stacked_bar_data.total, color = "red")

#Plot 2 - overlay - "bottom" series
bottom_plot = sns.barplot(x = stacked_bar_data.Group, y = stacked_bar_data.Series1, color = "#0000A3")


topbar = plt.Rectangle((0,0),1,1,fc="red", edgecolor = 'none')
bottombar = plt.Rectangle((0,0),1,1,fc='#0000A3',  edgecolor = 'none')
l = plt.legend([bottombar, topbar], ['Bottom Bar', 'Top Bar'], loc=1, ncol = 2, prop={'size':16})
l.draw_frame(False)

#Optional code - Make plot look nicer
sns.despine(left=True)
bottom_plot.set_ylabel("Y-axis label")
bottom_plot.set_xlabel("X-axis label")

#Set fonts to consistent 16pt size
for item in ([bottom_plot.xaxis.label, bottom_plot.yaxis.label] +
             bottom_plot.get_xticklabels() + bottom_plot.get_yticklabels()):
    item.set_fontsize(16)

In [ ]:
# a = !ls *.dat
# for i in a:
#     df = pd.read_csv(i)
#     try:
#             print(df.group_id.count(),'||||||||||||',i)
#     except:
#         pass
df = pd.read_csv("Y-15-[2-100]-rating==5-groups.dat")
df

In [ ]:
# Import Data
df = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/mpg_ggplot2.csv")
df_counts = df.groupby(['hwy', 'cty']).size().reset_index(name='counts')

# Draw Stripplot
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)    
sns.stripplot(df_counts.cty, df_counts.hwy, size=df_counts.counts*2, ax=ax)

# Decorations
plt.title('Counts Plot - Size of circle is bigger as more points overlap', fontsize=22)
plt.show()

In [ ]:
df.groupby(['hwy', 'cty']).size().reset_index(name='counts').head()

In [ ]:
df.head()

In [ ]:
import requests, json
!pip install plotly
import plotly.graph_objects as go

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
data = requests.get(url).json()

In [ ]:
data

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 1,
      thickness = 1,
      line = dict(color = "black", width = 0.5),
      label = df.group,
      color = "blue"
    ),
    link = dict(
      source = a.level_0, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = a.level_1,
      value = a[0]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=1)
fig.show()

In [ ]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Nodes & links
nodes = [['ID', 'Label', 'Color'],
        [0,'AKJ Education','#4994CE'],
        [1,'Amazon','#8A5988'],
        [2,'Flipkart','#449E9E'],
        [3,'Books','#7FC241'],
        [4,'Computers & tablets','#D3D3D3'],
        [5,'Other','#4994CE'],]

# links with your data
links = [['Source','Target','Value','Link Color'],

        # AKJ
        [0,3,1,'rgba(127, 194, 65, 0.2)'],
        [0,4,1,'rgba(127, 194, 65, 0.2)'],

        # Amazon
        [1,3,1,'rgba(211, 211, 211, 0.5)'],
        [1,4,1,'rgba(211, 211, 211, 0.5)'],
        [1,5,1,'rgba(211, 211, 211, 0.5)'],

        # Flipkart
        [2,5,1,'rgba(253, 227, 212, 1)'],
        [2,3,1,'rgba(253, 227, 212, 1)'],]


# Retrieve headers and build dataframes
nodes_headers = nodes.pop(0)
links_headers = links.pop(0)
df_nodes = pd.DataFrame(nodes, columns = nodes_headers)
df_links = pd.DataFrame(links, columns = links_headers)

# Sankey plot setup
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
    # thickness = 30,
      line = dict(
        color = "black",
        width = 0
      ),
      label =  df_nodes['Label'].dropna(axis=0, how='any'),
      color = df_nodes['Color']
    ),
    link = dict(
      source = df_links['Source'].dropna(axis=0, how='any'),
      target = df_links['Target'].dropna(axis=0, how='any'),
      value = df_links['Value'].dropna(axis=0, how='any'),
      color = df_links['Link Color'].dropna(axis=0, how='any'),
  )
)

layout = dict(
        title = "Draw Sankey Diagram from dataframes",
    height = 772,
    font = dict(
      size = 10),)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)